# Bell & Lin Opacities

Testing the implementation of the Bell & Lin 1994 opacities and comparing against the orignal paper and the Rice & Armitage 2004 figure.

In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.pyplot import style
style.use(['seaborn-darkgrid',{'font.size':10}]);
sys.path.append(os.path.abspath('../semenov_opacity/'))
import semenov_opacs as so

In [ ]:
def kappa_bell_lin_94(T,rho,return_all=False):
    """
    Calculate Bell & Lin 1994 opacities.
    
         **** NOTE ****
       
    THIS STILL NEEDS TESTING!
    
    Arguments:
    ----------
    
    T : float | array
    : temperatures at which to return the opacity
    
    rho : float
    : density for which to return the opacity
    
    Keywords:
    ---------
    
    return_all : bool
    : if true:  return the opacities of all the species
      if false: return only the total opacity
      
    Output:
    -------
    
    kappa : array
    : the opacity at the given temperatures and density
    """
    
    print('         **** NOTE ****\n')
    print('    THIS STILL NEEDS TESTING!\n')
    print('             *******\n')
    
    kappa_table = np.array([
        [2e-4,      0.0,   2.0],
        [2e16,      0.0,  -7.0],
        [0.1,       0.0,   0.5],
        [2e81,      1.0, -24.0],
        [1e-8,   2./3.0,   3.0],
        [1e-36,  1./3.0,  10.0],
        [1.5e20,    1.0,  -2.5],
        [0.348,     0.0,   0.0]])

    n_s     = len(kappa_table)
    T_table = np.logspace(2,6,500)
    kappa   = np.array([k*rho**a*T_table**b for k,a,b in kappa_table])
    i_trans = [abs(kappa[i]-kappa[i+1]).argmin() for i in range(n_s-1)]

    k_bl    = np.zeros(T_table.shape)
    i_s     = 0
    for i in range(len(T_table)):

        # switch to next species

        if i_s<n_s-1 and i>i_trans[i_s]: i_s+=1

        # pick opacity
            
        k_bl[i] = kappa[i_s,i]
    
    if return_all:
        return np.array([np.interp(T,T_table,k) for k in kappa])
    else:
        return np.interp(T,T_table,k_bl)

## Compare to Semenov et al. 2003

In [ ]:
temp    = np.logspace(1,6,300)

for rho in 10.**np.arange(-5,-10,-1):
    
    f,ax    = plt.subplots()

    #kappas = kappa_bell_lin_94(temp,rho,return_all=True)
    k_bl   = kappa_bell_lin_94(temp,rho)
    
    t_s,k_s = so.get_opac('nrm','c','s',True,rho,len(temp),temp[0],temp[-1])

    #f,ax = plt.subplots()
    #for i,k in enumerate(kappas):
    #    ax.loglog(temp,k,color=plt.cm.get_cmap('Reds')(1-i/(len(kappas)-1.)))

    ax.loglog(t_s,k_s,'-',lw=2)
    ax.loglog(temp,k_bl,'--',lw=2)
    ax.set_ylim(1e-5,1e5);